# LOADING META-DATA IN CSV FILE

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/05-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/05-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/05-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA Server
We are reusing the start server notebook for launching the EVA server

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File '00-start-eva-server.ipynb' already there; not retrieving.



Note: you may need to restart the kernel to use updated packages.


nohup eva_server > eva.log 2>&1 &


Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

cursor = connection.cursor()

### Video Files
 - Download a subset of the Berkeley Deepdrive dataset from Dropbox

In [3]:
# sample dataset of 2 videos
!wget -nc https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip

# unzip
!unzip -n bddtest.zip

--2023-04-30 10:33:58--  https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: /s/raw/kg1q69ijbhjfecw/bddtest.zip [following]
--2023-04-30 10:33:58--  https://www.dropbox.com/s/raw/kg1q69ijbhjfecw/bddtest.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 

302 Found
Location: https://uc213815cfc2a467ae01520df8cb.dl.dropboxusercontent.com/cd/0/inline/B7KW8Cq6sQzqt4VZVA28HKdIlXBPclZNRVabe3OYTLHesBrbdwE_jxFaz3HoN71BdXXvRsxwq0I_34m0W-qq5ssQ_KtjgOsUX7QhOXUrPYuCWFXbqjR83OZcwUH7u9S8eMpkV4hHCDJfityElvUVX-wnCG_v9vaAONZA30h24nKj1w/file# [following]
--2023-04-30 10:33:59--  https://uc213815cfc2a467ae01520df8cb.dl.dropboxusercontent.com/cd/0/inline/B7KW8Cq6sQzqt4VZVA28HKdIlXBPclZNRVabe3OYTLHesBrbdwE_jxFaz3HoN71BdXXvRsxwq0I_34m0W-qq5ssQ_KtjgOsUX7QhOXUrPYuCWFXbqjR83OZcwUH7u9S8eMpkV4hHCDJfityElvUVX-wnCG_v9vaAONZA30h24nKj1w/file
Resolving uc213815cfc2a467ae01520df8cb.dl.dropboxusercontent.com (uc213815cfc2a467ae01520df8cb.dl.dropboxusercontent.com)... 

162.125.9.15, 2620:100:601f:15::a27d:90f
Connecting to uc213815cfc2a467ae01520df8cb.dl.dropboxusercontent.com (uc213815cfc2a467ae01520df8cb.dl.dropboxusercontent.com)|162.125.9.15|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: /cd/0/inline2/B7KHEXpaGoXeUzEg6e_onyJvMMrer5smSFKol1LDx9O4eJSQBFnpwjYBeCv95HjxPoT-G_pDOE-A7QMUEXXxfjSRmGMWuNeT6mAtvzGQhU4iTknLtatUODWO50xtRpVU84_I5neGv1CuKcc4BugDSvT7-tmXUkLyh0BLlTr9MMqDYQH7yPj8dioq9hCHMXmu6LLWr9p07bJEqjAqEmWez2FX1Jq32e-lVsMLkxGv7E4ig_ZTH8P7a2bBW5_BE4JD6K0JFjz1GCRsbGlvLg8P4jsFX1uxM2o8UKyEvXHDoCf7Vis3I1j-WrzMcHdV-nByTzm3kVNPMkebcMTbYKcfD-F1k8Fteg_eeq0luM9ysdmp5TUtWEX2BU0UQGml7KEdDgOUWxn0qbJnPVlLscWvOhnxGWLy6NxNw-cYJY8I3xbaPw/file [following]
--2023-04-30 10:34:00--  https://uc213815cfc2a467ae01520df8cb.dl.dropboxusercontent.com/cd/0/inline2/B7KHEXpaGoXeUzEg6e_onyJvMMrer5smSFKol1LDx9O4eJSQBFnpwjYBeCv95HjxPoT-G_pDOE-A7QMUEXXxfjSRmGMWuNeT6mAtvzGQhU4iTknLtatUODWO50xtRpVU84_I5neGv1CuKcc4BugDSvT7-tmXUkLyh0BLlTr9MMqDYQH7yPj8dioq9hCHMXmu6LLWr9p07bJEqjAqEmWez2FX1Jq32e-lVsMLkxGv7E4ig_ZTH8P7a2bBW5_BE4JD6K0JFjz1GCRsbGlvLg8P4jsFX1uxM2o8UKyEvXHDoCf7Vis3I1j-WrzMcHdV-nByTzm3kVNPMkebcMTbYKcfD-F1k8Fteg_eeq0luM9ysdmp5TUtWEX2BU0UQGml7KEdDgOUWxn0qbJnPVlLscWvOhnxGWLy6NxNw

...


  inflating: bddtest/videos/bddtest_00a04f658c891f94.mp4  


  inflating: bddtest/videos/bddtest_00a820ef2b98dcf5.mp4  


  inflating: bddtest/videos/bddtest_00a395fed60c0b47.mp4  


   creating: bddtest/info/
  inflating: bddtest/info/bddtest_00a04f65af2ab984.csv  
  inflating: bddtest/info/bddtest_00a9cd6bb39be004.csv  
  inflating: bddtest/info/bddtest_00a2e3ca5c856cde.csv  
  inflating: bddtest/info/bddtest_00a04f658c891f94.csv  
  inflating: bddtest/info/bddtest_00a2e3ca62992459.csv  
  inflating: bddtest/info/bddtest_00a0f008a315437f.csv  


  inflating: bddtest/info/bddtest_00a0f0083c67908e.csv  
  inflating: bddtest/info/bddtest_00a820ef2b98dcf5.csv  
  inflating: bddtest/info/bddtest_00a2f5b6d4217a96.csv  
  inflating: bddtest/info/bddtest_00a395fed60c0b47.csv  


## Load 2 BDD videos

In [4]:
cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4" INTO bddtest_1;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a395fed60c0b47.mp4" INTO bddtest_2;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded VIDEO: 1
@query_time: 0.10894125513732433
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded VIDEO: 1
@query_time: 0.08081590756773949


## Create a BDD Meta-data Table

In [5]:
create_table_query = f""" 

    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.execute(create_table_query)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.061140066012740135


## Load the 2 corresponding CSV files associated with the 2 previously loaded videos

In [6]:
cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                          CSV  Number of loaded frames
0  bddtest/info/bddtest_00a2e3ca5c856cde.csv                     3401
@query_time: 0.48897757567465305


@status: ResponseStatus.SUCCESS
@batch: 
                                          CSV  Number of loaded frames
0  bddtest/info/bddtest_00a395fed60c0b47.csv                     4355
@query_time: 0.5176386311650276


## Run a Query over the Video dataset


In [7]:
cursor.execute("""SELECT id, FastRCNNObjectDetector(data) 
                  FROM bddtest_1 
                  WHERE id < 3;""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
    bddtest_1.id   
0             0  \
1             1   
2             2   

                                                                         fastrcnnobjectdetector.labels   
0  [stop sign, car, stop sign, car, car, car, car, car, car, car, car, stop sign, car, car, car, ca...  \
1  [stop sign, car, car, stop sign, car, car, car, car, car, car, car, car, stop sign, stop sign, c...   
2   [stop sign, car, car, stop sign, car, car, car, car, car, car, car, stop sign, car, car, car, car]   

                                                                         fastrcnnobjectdetector.bboxes   
0  [[343.09747 195.47418 405.7211  258.95984], [675.4714  326.11798 751.1334  368.09262], [ 954.581...  \
1  [[334.63928 191.12814 398.79654 254.96082], [484.48923 332.7234  717.053   516.7328 ], [673.2345...   
2  [[327.6995  189.36577 392.4098  253.57695], [670.9204  324.10666 750.4485  367.99106], [482.2456...   

                               

## Run a query over the Meta-data Table

In [8]:
cursor.execute("""SELECT *
                  FROM bddtestmeta
                  WHERE bddtestmeta.video_id = 4;""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
       bddtestmeta._row_id  bddtestmeta.id  bddtestmeta.frame_id   
0                       1            8185                     0  \
1                       2            8186                     0   
2                       3            8187                     0   
3                       4            8188                     0   
4                       5            8189                     0   
...                   ...             ...                   ...   
3396                 3397           11581                  1197   
3397                 3398           11582                  1197   
3398                 3399           11583                  1197   
3399                 3400           11584                  1197   
3400                 3401           11585                  1197   

      bddtestmeta.video_id bddtestmeta.dataset_name bddtestmeta.label   
0                        4                  bddtest               car  \
1      